In [1]:
from pyecharts.charts import Bar, Calendar, Pie
from pyecharts.globals import CurrentConfig, NotebookType
from pyecharts import options as opts
from pyecharts.globals import ThemeType
from snapshot_selenium import snapshot
from pyecharts.render import make_snapshot

In [2]:
import json
from datetime import datetime
from dateutil import parser
from urllib.parse import urlparse

In [3]:
with open("aw-buckets-export_phone.json", "r") as f:
    data = json.load(f)

In [6]:
total_duration = 0.0
app_usage = {}
dayrecord = {}

for event in data['buckets']['aw-watcher-android-test']['events']:
    date = parser.parse(event["timestamp"]).strftime("%Y-%m-%d")

    if '2022' in date:
        duration = event['duration']
        total_duration = duration + total_duration
        appname = event['data']['app']

        if appname in app_usage:
            app_usage[appname] = duration + app_usage[appname]
        else:
            app_usage[appname] = duration

        # if appname != 'LockApp.exe':
        if date in dayrecord:
            dayrecord[date] += duration
        else:
            dayrecord[date] = duration


In [9]:
hours = total_duration//3600
print("Total phone Usage in hours: ", hours)
print("Total phone Usage in days: ", hours//24)

Total phone Usage in hours:  1038.0
Total phone Usage in days:  43.0


In [11]:
app_usage_hourly = {}

for k,v in app_usage.items():
    usage_time = int(v//3600)
    if usage_time > 10:
        app_usage_hourly[k] = usage_time

print(app_usage_hourly)

{'Nova7': 39, 'WhatsApp': 40, 'Telegram': 96, 'Firefox': 169, 'Clock': 12, 'Daylio': 26, 'Amazon': 12, 'Duolingo': 21, 'YouTube': 254, 'Phone': 19, 'Boost': 73, 'Dumbbell Workout at Home': 17, 'Lose Weight App for Men': 11, 'Twitter': 21, 'Joey': 11}


In [16]:
pie = Pie(init_opts=opts.InitOpts(theme=ThemeType.CHALK))

t = [list(z) for z in zip(app_usage_hourly.keys(), app_usage_hourly.values())]
pie.add("", t)
pie.set_global_opts(legend_opts=opts.LegendOpts(is_show=False), title_opts=opts.TitleOpts(title="Phone App Usage (in hours)",  pos_left="right",))
pie.set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
# pie.render()
make_snapshot(snapshot, cal.render(), 'phone_app_usage.png')

In [13]:
dataToPlot = []

for k,v in dayrecord.items():
    date_ = datetime.strptime(k, "%Y-%m-%d")
    if date_.year == 2022:
        dataToPlot.append([date_.strftime('%Y-%m-%d'),int(v)//3600])


In [14]:
cal = Calendar()
cal.add("", dataToPlot, calendar_opts=opts.CalendarOpts(range_="2022"))
cal.set_global_opts(
        title_opts=opts.TitleOpts(title="Phone Usage", subtitle="Hours per day"),
        visualmap_opts=opts.VisualMapOpts(
            max_=40,
            min_=0,
            orient="horizontal",
            is_piecewise=True,
            pos_top="230px",
            pos_left="100px",
        ),
    )
cal.render()

# make_snapshot(snapshot, cal.render(), 'PC_usage_heatmap.png')

'/home/pwnbox/dev/year-in-review/appusage/render.html'